In [1]:
%load_ext autoreload
%autoreload 2

# Goal: Run facility choice causal model optimization to find input parameters for Vivarium

The optimization results are output in a `.csv` file at the end of the notebook.

## Requirements:

If you want to run this notebook or any of the code in the
`birth_facility.py` or `solution_finding.py` modules, you will need the right environment.
You can create it with these steps (on 64-bit Linux):

- `conda create --name facility_choice --file environment_lock_conda.txt`
- `conda activate facility_choice`
- `pip install -r environment_lock_pip.txt`

If you're not on 64-bit Linux, replacing the first command with `conda create --name facility_choice python=3.10`
should approximately recreate the environment.

If you want to update all the packages in the environment, create it using `conda env create --name facility_choice --file environment.yaml` (no need for any `pip` commands).
Then, run the following commands inside the environment you created to update the lock files:

- `conda list --explicit > environment_lock_conda.txt`
- `pip freeze | grep -v 'file:///' | grep -v '\-e' > environment_lock_pip.txt`

You will also need to be on the IHME cluster to run this notebook, as the causal model
nanosim loads LBWSG data from an Artifact for the MNCNH Portfolio model.

In [1]:
import pandas as pd

import birth_facility as bf
import solution_finding as sf

!date
!whoami
!uname -a
!pwd
!python --version
!conda info --envs | grep '\*'
!conda list | grep -e pandas -e numpy -e scipy -e statsmodels -e matplotlib -e gbd

Thu Dec 11 12:34:06 PST 2025
tylerdy
Linux long-slurm-sarchive-p0116 5.4.0-216-generic #236-Ubuntu SMP Fri Apr 11 19:53:21 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/homes/tylerdy/vivarium_gates_mncnh/src/vivarium_gates_mncnh/data/facility_choice
Python 3.10.19
facility_choice      * /ihme/homes/tylerdy/miniforge3/envs/facility_choice
gbd                       4.37.1                   pypi_0    pypi
gbd-mapping               5.0.0                    pypi_0    pypi
matplotlib-inline         0.1.7                    pypi_0    pypi
numpy                     1.26.4                   pypi_0    pypi
pandas                    1.5.3                    pypi_0    pypi
pandas-stubs              2.2.3.250308             pypi_0    pypi
scipy                     1.15.3                   pypi_0    pypi
statsmodels               0.14.5                   pypi_0    pypi


# Load data

The input data comes from two sources:

- An artifact for the `vivarium_gates_mncnh` simulation, which is used for population structure, LBWSG exposure and RRs, ANC and IFD coverage, etc.
- The file `facility_choice_data.xlsx` in this folder (downloaded from
  the [master version on
  Sharepoint](https://uwnetid.sharepoint.com/:x:/r/sites/ihme_simulation_science_team/Shared%20Documents/Research/BMGF_MNCH/MNCNH%20portfolio%20products/01_Planning/facility_choice_data.xlsx?d=wf3b3dd5f641f413ba3537c4ca3364cdf&csf=1&web=1&e=cNf9X7)),
  which contains all the other country-specific data to run the model

In [3]:
data_e = bf.BirthFacilityChoiceData('ethiopia')
data_n = bf.BirthFacilityChoiceData('nigeria')
data_p = bf.BirthFacilityChoiceData('pakistan')

# Create models for all 3 countries

Each of the three model objects defines the causal model nanosim that
will be run repeatedly by the `OptimalSolutionFinder.find_solutions`
optimization routine below.

In [4]:
# Choose a population size that's large enough to get stochastically
# stable results, but small enough that the optimization isn't too slow
pop_size = 100_000
# In Vivarium, we will be using the 2-facility-type model (at-home vs.
# in-facility)
num_facility_types = 2

# The random seeds were generated by calling numpy.random.SeedSequence()
# and then copying the generated entropy
model_e = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_e, pop_size, num_facility_types,
    seed=97269740763240770980627172105157383126
)
model_n = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_n, pop_size, num_facility_types,
    seed=230426788122306909803111949239871602900
)
model_p = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_p, pop_size, num_facility_types,
    seed=150031875166580400207947577126615739298
)

# Create solution finder objects and specify inputs for optimization

In [5]:
# These are the 5 parameters we need to solve for in the optimization
model_e.parameter_data

attribute,correlated_pair,bounds,position
parameter_name,,,
"corr(anc, lbwsg_category)","(anc, lbwsg_category)","(-1, 1)",0
"corr(anc, facility)","(anc, facility)","(-1, 1)",1
"corr(lbwsg_category, facility)","(lbwsg_category, facility)","(-1, 1)",2
prob_home_given_believed_preterm,<NA>,"(0, 1)",3
prob_home_given_believed_term,<NA>,"(0, 1)",4


In [6]:
finder_e = sf.OptimalSolutionFinder(model_e)
finder_n = sf.OptimalSolutionFinder(model_n)
finder_p = sf.OptimalSolutionFinder(model_p)

# 25 initial conditions seems to be sufficient to get close to the
# global minimum
n_points = 25
# Fix the two LBWSG correlations at 0.2 -- According to the above
# parameter_data dataframe, these parameters are in positions 0 and 2 in
# the parameter vector
fixed_x_components = {0: 0.2, 2: 0.2}

# Find solutions

# Ethiopia

In [7]:
%%time
solutions_e = finder_e.find_solutions(n_points, fixed_x_components)
print(solutions_e.sorted_losses)
solutions_e.sorted_x_values

[4.8063186319780016e-08, 5.442511086251045e-08, 7.756449049711733e-08, 8.697127296120044e-08, 9.69375432147146e-08, 1.0015761786608124e-07, 1.1445639569451771e-07, 1.1472727579064923e-07, 1.428473973552613e-07, 1.4578752627336655e-07, 1.4891242572367958e-07, 3.445635529120139e-07, 3.8091918086458065e-07, 3.826816382668241e-07, 3.8981796879333785e-07, 4.2598658722692306e-07, 4.5093037692733873e-07, 0.013065316148598627, 0.016329086314096575, 0.06041710589752469, 0.07279389610368248, 0.09400056614610885, 0.11729484195104278, 0.3407973385487314, inf]
CPU times: user 1h 20min 47s, sys: 7h 5min 45s, total: 8h 26min 32s
Wall time: 7min 58s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.677202,0.2,0.319683,0.476647
1,0.2,0.677122,0.2,0.319563,0.476826
2,0.2,0.678338,0.2,0.318958,0.476341
3,0.2,0.678382,0.2,0.318756,0.476445
4,0.2,0.676870,0.2,0.320218,0.476417
5,0.2,0.677361,0.2,0.318991,0.476976
6,0.2,0.678091,0.2,0.319552,0.476563
7,0.2,0.678702,0.2,0.318470,0.476492
8,0.2,0.678527,0.2,0.317678,0.476840
9,0.2,0.677366,0.2,0.318699,0.476688


In [8]:
data_e.targets_2_facility_types.to_frame().join(solutions_e.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.183369  0.182788  0.183754   
               anc1                     0.664786  0.664327  0.664357   
preterm        at_home                  0.163579  0.163849  0.163394   
               in_facility              0.163486  0.162491  0.163888   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.183377  0.183377  0.183016  0.183945  0.181793   
               anc1         0.665117  0.664707  0.664404  0.664015  0.665113   
preterm        at_home      0.163454  0.163676  0.164003  0.163970  0.163725   
               in_facility  0.163840  0.163773  0.162387  0.163505  0.162607   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.182266  0.182055  ...  0.183427  0.182038   
               anc1         0.664942  0.664720  ...  0.664160  0.664622   
preterm        at_home      0.163673  0.163345  ...  0.163490  0.163312   
               in_facility  0.162631  0.162906  ...  0.162810  0.162986   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.181959  0.125684  0.236363  0.166484  0.181147   
               anc1         0.749910  0.716588  0.832056  0.524399  0.604225   
preterm        at_home      0.089282  0.081970  0.373598  0.279622  0.323625   
               in_facility  0.208392  0.220916  0.074284  0.019801  0.010931   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.159096  0.704992  0.183754  
               anc1         0.408694  0.781019  0.664357  
preterm        at_home      0.245508  0.643139  0.163394  
               in_facility  0.010116  0.014930  0.163888  

[4 rows x 26 columns]

In [9]:
data_e.input_probabilities.to_frame().join(solutions_e.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.515312,0.516510,0.512060,0.512060,0.515200,0.516510,0.515200,0.51651,0.516510,0.516510,...,0.51651,0.51651,0.512060,0.512060,0.512060,0.515200,0.515200,0.512060,0.515200,0.512060
Female,0.484688,0.483490,0.487940,0.487940,0.484800,0.483490,0.484800,0.48349,0.483490,0.483490,...,0.48349,0.48349,0.487940,0.487940,0.487940,0.484800,0.484800,0.487940,0.484800,0.487940
anc0,0.219890,0.219270,0.219260,0.219220,0.219220,0.219270,0.219250,0.21926,0.219240,0.219220,...,0.21927,0.21924,0.219720,0.219440,0.219620,0.219240,0.219380,0.219490,0.220570,0.219260
anc1,0.780110,0.780730,0.780740,0.780780,0.780780,0.780730,0.780750,0.78074,0.780760,0.780780,...,0.78073,0.78076,0.780280,0.780560,0.780380,0.780760,0.780620,0.780510,0.779430,0.780740
preterm,0.163527,0.163090,0.163670,0.163670,0.163730,0.163100,0.163710,0.16310,0.163090,0.163100,...,0.16311,0.16313,0.163740,0.163520,0.163710,0.163760,0.163710,0.162200,0.163030,0.163670
term,0.836473,0.836910,0.836330,0.836330,0.836270,0.836900,0.836290,0.83690,0.836910,0.836900,...,0.83689,0.83687,0.836260,0.836480,0.836290,0.836240,0.836290,0.837800,0.836970,0.836330
at_home,0.441073,0.441260,0.441020,0.440490,0.440810,0.441150,0.441240,0.44086,0.440880,0.441090,...,0.44125,0.44118,0.374880,0.413080,0.298770,0.554070,0.488590,0.646090,0.235750,0.441020
in_facility,0.558927,0.558740,0.558980,0.559510,0.559190,0.558850,0.558760,0.55914,0.559120,0.558910,...,0.55875,0.55882,0.625120,0.586920,0.701230,0.445930,0.511410,0.353910,0.764250,0.558980
BEmONC,0.089922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Nigeria

In [10]:
%%time
solutions_n = finder_n.find_solutions(n_points, fixed_x_components)
print(solutions_n.sorted_losses)
solutions_n.sorted_x_values

[1.3365739792714493e-07, 1.3945360488598624e-07, 1.4409506920998183e-07, 1.5140105202871013e-07, 1.5835208244663335e-07, 1.646486973427841e-07, 1.7671979746225475e-07, 1.7893387327383437e-07, 1.8284013680869293e-07, 1.850635515943111e-07, 1.8545801583336186e-07, 1.8595322892611676e-07, 1.865681072965586e-07, 2.5530922487249086e-07, 2.5577577567048593e-07, 3.3468042814899235e-07, 5.550792228081747e-07, 7.704973784550262e-07, 1.2089978171925253e-06, 0.017333310003975644, 0.021682840030201378, 0.02237575021075011, 0.022401284807174293, 0.0277732697297568, 0.3042507132184108]
CPU times: user 1h 17min 28s, sys: 6h 45min 58s, total: 8h 3min 27s
Wall time: 7min 35s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.412383,0.2,0.391762,0.512343
1,0.2,0.412283,0.2,0.392108,0.512281
2,0.2,0.412452,0.2,0.391503,0.512214
3,0.2,0.412450,0.2,0.392187,0.512076
4,0.2,0.412392,0.2,0.392149,0.512027
5,0.2,0.412545,0.2,0.391432,0.512163
6,0.2,0.412401,0.2,0.391563,0.512452
7,0.2,0.412647,0.2,0.392036,0.511990
8,0.2,0.411610,0.2,0.391338,0.512021
9,0.2,0.411153,0.2,0.392499,0.511773


In [11]:
data_n.targets_2_facility_types.to_frame().join(solutions_n.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.309069  0.309111  0.309863   
               anc1                     0.584168  0.584144  0.583867   
preterm        at_home                  0.172663  0.172881  0.173309   
               in_facility              0.153000  0.153326  0.153349   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.308411  0.308360  0.308474  0.308477  0.308384   
               anc1         0.584521  0.584502  0.584398  0.584556  0.584257   
preterm        at_home      0.172711  0.172885  0.173138  0.172684  0.172640   
               in_facility  0.153491  0.153325  0.153064  0.153518  0.153531   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.309851  0.309162  ...  0.309435  0.309243   
               anc1         0.584050  0.583665  ...  0.583894  0.584523   
preterm        at_home      0.173314  0.173643  ...  0.173914  0.172539   
               in_facility  0.153329  0.153077  ...  0.152850  0.153759   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.308450  0.308318  0.440322  0.463979  0.445632   
               anc1         0.583256  0.583375  0.459063  0.449406  0.432139   
preterm        at_home      0.172751  0.172916  0.185839  0.164210  0.191906   
               in_facility  0.153399  0.153752  0.136001  0.161539  0.126673   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.454171  0.306629  0.687010  
               anc1         0.434822  0.673181  0.892133  
preterm        at_home      0.162620  0.062131  0.681957  
               in_facility  0.164325  0.235975  0.063670  

[4 rows x 26 columns]

In [12]:
data_n.input_probabilities.to_frame().join(solutions_n.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.512450,0.510960,0.512480,0.510960,0.51096,0.510960,0.51096,0.510960,0.512480,0.509000,...,0.509000,0.510960,0.510960,0.509000,0.510960,0.512480,0.511540,0.512480,0.512480,0.509000
Female,0.487550,0.489040,0.487520,0.489040,0.48904,0.489040,0.48904,0.489040,0.487520,0.491000,...,0.491000,0.489040,0.489040,0.491000,0.489040,0.487520,0.488460,0.487520,0.487520,0.491000
anc0,0.257001,0.256930,0.256920,0.256930,0.25694,0.256910,0.25694,0.256920,0.256930,0.256920,...,0.256920,0.256950,0.256930,0.256910,0.256290,0.256240,0.257230,0.257260,0.257510,0.257420
anc1,0.742999,0.743070,0.743080,0.743070,0.74306,0.743090,0.74306,0.743080,0.743070,0.743080,...,0.743080,0.743050,0.743070,0.743090,0.743710,0.743760,0.742770,0.742740,0.742490,0.742580
preterm,0.162567,0.162840,0.163060,0.162840,0.16284,0.162830,0.16284,0.162830,0.163050,0.163090,...,0.163100,0.162890,0.162830,0.163090,0.163200,0.163000,0.163490,0.163370,0.162750,0.163010
term,0.837433,0.837160,0.836940,0.837160,0.83716,0.837170,0.83716,0.837170,0.836950,0.836910,...,0.836900,0.837110,0.837170,0.836910,0.836800,0.837000,0.836510,0.836630,0.837250,0.836990
at_home,0.486533,0.486520,0.486530,0.486420,0.48645,0.486490,0.48638,0.486620,0.486400,0.486860,...,0.486620,0.486210,0.487350,0.487290,0.545740,0.546860,0.564390,0.560200,0.421210,0.160670
in_facility,0.513467,0.513480,0.513470,0.513580,0.51355,0.513510,0.51362,0.513380,0.513600,0.513140,...,0.513380,0.513790,0.512650,0.512710,0.454260,0.453140,0.435610,0.439800,0.578790,0.839330
BEmONC,0.002271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pakistan

In [13]:
%%time
solutions_p = finder_p.find_solutions(n_points, fixed_x_components)
print(solutions_p.sorted_losses)
solutions_p.sorted_x_values

[9.756016537965095e-07, 1.1082569258968888e-06, 1.1559449224485263e-06, 1.1700151479887566e-06, 1.2214843183588897e-06, 1.2505301766330135e-06, 1.2554217706517434e-06, 1.2808090386062787e-06, 1.4144544472349807e-06, 1.535753566872522e-06, 1.5770721220054185e-06, 1.8179243158611413e-06, 2.0916681225946476e-06, 2.12340041805259e-06, 4.656580713469083e-06, 0.00013603467784251144, 0.002936665592967458, 0.0037663846738673623, 0.004948802503582095, 0.005205991073177985, 0.006030508219957764, 0.010907898072460731, 0.0963816957470599, 0.10821811341320287, inf]
CPU times: user 1h 10min 33s, sys: 6h 11min 34s, total: 7h 22min 7s
Wall time: 6min 56s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.348486,0.2,0.172872,0.239061
1,0.2,0.346198,0.2,0.173244,0.239029
2,0.2,0.346855,0.2,0.173085,0.238796
3,0.2,0.348177,0.2,0.172853,0.239208
4,0.2,0.345465,0.2,0.173431,0.239182
5,0.2,0.347054,0.2,0.172828,0.238754
6,0.2,0.347432,0.2,0.172787,0.238696
7,0.2,0.347584,0.2,0.172734,0.238756
8,0.2,0.348106,0.2,0.172465,0.238693
9,0.2,0.348451,0.2,0.172544,0.238394


In [14]:
data_p.targets_2_facility_types.to_frame().join(solutions_p.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.556322  0.555275  0.555275   
               anc1                     0.795407  0.795315  0.795575   
preterm        at_home                  0.194529  0.193895  0.195722   
               in_facility              0.155164  0.153850  0.153327   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.557327  0.556146  0.558523  0.556663  0.556515   
               anc1         0.795670  0.795328  0.795119  0.795789  0.795558   
preterm        at_home      0.195303  0.195087  0.194126  0.194051  0.193940   
               in_facility  0.153447  0.153616  0.154147  0.154180  0.153763   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.556456  0.556013  ...  0.564643  0.624166   
               anc1         0.795506  0.796042  ...  0.785676  0.783440   
preterm        at_home      0.193896  0.193901  ...  0.200875  0.172874   
               in_facility  0.153773  0.154284  ...  0.151370  0.160931   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.574136  0.562120  0.554717  0.649493  0.458836   
               anc1         0.721545  0.866662  0.868784  0.774182  0.874432   
preterm        at_home      0.195544  0.205525  0.186576  0.160699  0.197622   
               in_facility  0.152287  0.154845  0.158499  0.166898  0.156990   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.722435  0.785337  0.556456  
               anc1         0.951659  0.693995  0.795506  
preterm        at_home      0.473516  0.463874  0.193896  
               in_facility  0.140715  0.035311  0.153773  

[4 rows x 26 columns]

In [15]:
data_p.input_probabilities.to_frame().join(solutions_p.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.519811,0.519420,0.515460,0.515460,0.51546,0.51732,0.517320,0.51942,0.519420,0.517320,...,0.519420,0.517320,0.51732,0.517320,0.515460,0.519420,0.515460,0.517320,0.518670,0.519420
Female,0.480189,0.480580,0.484540,0.484540,0.48454,0.48268,0.482680,0.48058,0.480580,0.482680,...,0.480580,0.482680,0.48268,0.482680,0.484540,0.480580,0.484540,0.482680,0.481330,0.480580
anc0,0.075919,0.075260,0.075260,0.075270,0.07525,0.07527,0.075270,0.07529,0.075280,0.075250,...,0.074950,0.074900,0.07493,0.075660,0.075570,0.074920,0.076280,0.075730,0.075700,0.075280
anc1,0.924081,0.924740,0.924740,0.924730,0.92475,0.92473,0.924730,0.92471,0.924720,0.924750,...,0.925050,0.925100,0.92507,0.924340,0.924430,0.925080,0.923720,0.924270,0.924300,0.924720
preterm,0.163933,0.162770,0.162760,0.162750,0.16285,0.16305,0.163040,0.16270,0.162700,0.163080,...,0.162800,0.163660,0.16481,0.162770,0.162850,0.165440,0.163380,0.162580,0.163490,0.162700
term,0.836067,0.837230,0.837240,0.837250,0.83715,0.83695,0.836960,0.83730,0.837300,0.836920,...,0.837200,0.836340,0.83519,0.837230,0.837150,0.834560,0.836620,0.837420,0.836510,0.837300
at_home,0.222744,0.222750,0.222510,0.222270,0.22267,0.22269,0.222210,0.22244,0.222490,0.222020,...,0.230890,0.228490,0.28950,0.156380,0.154950,0.235160,0.157270,0.065700,0.299090,0.222490
in_facility,0.777256,0.777250,0.777490,0.777730,0.77733,0.77731,0.777790,0.77756,0.777510,0.777980,...,0.769110,0.771510,0.71050,0.843620,0.845050,0.764840,0.842730,0.934300,0.700910,0.777510
BEmONC,0.264677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Collect optimization results from the 3 countries

Select the solution with the lowest log loss for each country. The above
solutions dataframes are already sorted by log loss, so we just select
the first row of each.

In [16]:
solutions_by_location = {
    'Ethiopia': solutions_e,
    'Nigeria': solutions_n,
    'Pakistan': solutions_p,
}
best_solutions = pd.concat({
    location: solutions.sorted_x_values.loc[0]
    for location, solutions in solutions_by_location.items()
}, axis=1)
best_solutions

,Ethiopia,Nigeria,Pakistan
parameter_name,,,
"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
"corr(anc, facility)",0.677202,0.412383,0.348486
"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
prob_home_given_believed_preterm,0.319683,0.391762,0.172872
prob_home_given_believed_term,0.476647,0.512343,0.239061


# Save results to file

In [17]:
output_filename = 'facility_choice_optimization_results.csv'
best_solutions.to_csv(output_filename)
pd.read_csv(output_filename)

,parameter_name,Ethiopia,Nigeria,Pakistan
0,"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
1,"corr(anc, facility)",0.677202,0.412383,0.348486
2,"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
3,prob_home_given_believed_preterm,0.319683,0.391762,0.172872
4,prob_home_given_believed_term,0.476647,0.512343,0.239061
